In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.stats import poisson, uniform, expon, pareto
from scipy.optimize import minimize, fsolve
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# OLD OBJECTIVE

In [2]:
# NEGATIVE to maximize
def F(c, alpha, N):
    sorted_c = np.sort(c)
    subCs    = sorted_c[:N]
    x1s    = np.power(subCs, -alpha) / np.sum(np.array([np.power(subCs[i], -alpha) for i in range(len(subCs))]))
    val1s  = np.array([subCs[i]*np.power(x1s[i], 1+1/alpha) for i in range(len(subCs))])
    
    x2s    = np.power(c, -alpha) / np.sum(np.array([np.power(c[i], -alpha) for i in range(len(c))]))
    val2s  = np.array([c[i]*np.power(x2s[i], 1+1/alpha) for i in range(len(c))])
    
    num = np.sum(val1s)
    den = np.sum(val2s)
    
    return -num/den

In [6]:
alpha = 3.
R = 10
# N < R^{alpha-1}
n = 2
N = 1
start_pos = np.array([1, 1])
bnds = tuple((1,10) for x in start_pos)

res = minimize(
    F, 
    start_pos, 
    method      = 'SLSQP',
    args        = (alpha, N),
    bounds      = bnds, 
    tol=1e-16,
    options={'ftol':1e-16, 'maxiter':1000}
)
res.x, res.fun, (n/N)**(1/alpha)

(array([1., 1.]), -1.259921049894873, 1.2599210498948732)

# NEW OBJECTIVE

In [58]:
# NEGATIVE to maximize
def F(c, alpha, N):
    sorted_c = np.sort(c)
    subCs    = sorted_c[:N]
    x1s    = np.power(subCs, -alpha) / np.sum(np.array([np.power(subCs[i], -alpha) for i in range(len(subCs))]))
    val1s  = np.array([alpha * np.log(subCs[i])*x1s[i] + x1s[i] * np.log(x1s[i]) for i in range(len(subCs))])
    
    x2s    = np.power(c, -alpha) / np.sum(np.array([np.power(c[i], -alpha) for i in range(len(c))]))
    val2s  = np.array([alpha * np.log(c[i])*x2s[i] + x2s[i] * np.log(x2s[i]) for i in range(len(c))])
    
    num = np.sum(val1s)
    den = np.sum(val2s)
    
    return -num/den

In [54]:
0.5 * np.log(0.5) + np.log(1.41413615)

-5.4740312186774e-05

In [56]:
0.5 * np.log(0.5), np.log(1.41413615)

(-0.34657359027997264, 0.34651884996778587)

In [ ]:
find a cost such that the allocation matches the log of cost

In [43]:
np.log(1)

0.0

In [61]:
alpha = 2.
R = 10
# N < R^{alpha-1}
n = 5
N = 3
start_pos = np.array([8, 8, 8, 8, 8])
bnds = tuple((1,10) for x in start_pos)

res = minimize(
    F, 
    start_pos, 
    method      = 'SLSQP',
    args        = (alpha, N),
    bounds      = bnds, 
    tol=1e-16,
    options={'ftol':1e-16, 'maxiter':1000}
)
res.x, res.fun, (n/N)**(1/alpha)

(array([2.10591685, 2.10591687, 2.10591687, 2.10591686, 3.20664703]),
 -483.33878969977815,
 1.2909944487358056)

In [68]:
F([2.10591685, 2.10591687, 2.10591687, 2.10591686, 3.20664703], alpha=2, N=3)

-483.33881139759245